In [1]:
!pip install boto3

In [2]:

import requests
import boto3
import json
import pandas as pd


In [3]:
weather_data_url = "https://api.openweathermap.org/data/2.5/weather"
api_key = "5a86031666828de795f0334121f9c1bb"   # Replace with your API key
city1 = "Tampa"  # Replace with desired city
city2 = "London"
bucket_name = "akabuck-1"  # Replace with your S3 bucket name
group_name = "Group_5"


In [4]:
def get_file_name(city,formatt):
    return group_name+"_weather_data_"+city+"."+formatt

In [5]:
def get_weather_data(city):
  
    params = {'q': city, 'appid': api_key}
    response = requests.get(weather_data_url, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception("Failed to fetch weather data")

In [6]:
def upload_to_s3(bucket_name, file_name, data):
    s3 = boto3.client('s3')
    s3.put_object(Bucket=bucket_name, Key=file_name, Body= data)

In [7]:

def get_weather_upload_s3(city,format="json"):
    
    try:
        weather_data_json = get_weather_data(city)
        if format.lower() == "csv":
            weather_data = get_csv_for_json(weather_data_json)
            file_name = get_file_name(city,"csv")
            
        else:
            weather_data = json.dumps(weather_data_json)
            file_name = get_file_name(city,"json")
            
        upload_to_s3(bucket_name, file_name, weather_data)
        print(f"Data uploaded successfully to S3 {bucket_name} with filename {file_name}")
    except Exception as e:
        print(f"An error occurred: {e}")
        
            

In [8]:
def get_csv_for_json (data_json):
    
    pd.set_option('display.max_columns', None)
    pd.set_option('display.max_rows', None)

    # Converting json to dataframe
    flattened_data = pd.json_normalize(data_json) # Flatten the JSON data

    # Flatten 'weather' column separately with prefix
    weather_data_norm = pd.json_normalize(data_json['weather']).add_prefix('weather.')

    # Merge the two DataFrames
    df = pd.concat([flattened_data, weather_data_norm], axis=1)
    df = df.drop(columns = ['weather'], axis = 1) # drop the weather column
    csv_content = df.to_csv(index=False)
    return csv_content.encode('utf-8')



In [9]:
get_weather_upload_s3(city1)#upload with json
get_weather_upload_s3(city2,"json")

Data uploaded successfully to S3 akabuck-1 with filename Group_5_weather_data_Tampa.json
Data uploaded successfully to S3 akabuck-1 with filename Group_5_weather_data_London.json


In [10]:
get_weather_upload_s3(city1,"csv")#upload with csv
get_weather_upload_s3(city2,"csv")

Data uploaded successfully to S3 akabuck-1 with filename Group_5_weather_data_Tampa.csv
Data uploaded successfully to S3 akabuck-1 with filename Group_5_weather_data_London.csv
